<a href="https://colab.research.google.com/github/EziamaUgonna/Bayesian_analysis-/blob/master/Deepcon1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # % tensorflow_version only exist in colab
  %tensorflow_version 2.
except Exception:
   pass

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2.`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
!pip install h5py pyyaml

In [3]:
# Packages
import numpy as np
from numpy.random import seed
seed(1)
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
#import vaex
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import tensorflow as tf
#tf.random.set_seed(123)
import tensorflow_probability as tfp
tfd = tfp.distributions
#from tensorflow_probability.python.math import random_rademacher
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization,concatenate,PReLU
from tensorflow.keras.layers import ReLU, Dropout,Flatten, Conv1D,MaxPooling1D,Reshape,Input,GaussianNoise
from tensorflow.keras.optimizers import Adam
# Settings
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import accuracy_score,confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve,classification_report,roc_auc_score
from imblearn.metrics import geometric_mean_score as gmean 
from imblearn.metrics import make_index_balanced_accuracy as iba
from imblearn.metrics import classification_report_imbalanced
from statsmodels.robust import mad
import pywt 
from scipy.signal import butter, deconvolve
from scipy import signal
sns.set()
from sklearn.preprocessing import MinMaxScaler
%config InlineBackend.figure_format = 'svg'
np.random.seed(12345)
tf.__version__

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be import

'2.1.0'

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/project_data/Bias_sensor1"

In [0]:
df1 = pd.read_csv('/content/drive/My Drive/project_data/Bias_sensor1/Sensor1_values_bias_0_1_dur_10_dep.csv').astype(np.float32)
df2 = pd.read_csv('/content/drive/My Drive/project_data/Bias_sensor1/Ground_truth_bias_0_1_dur_10_sensor1_dep_train.csv')
df1 = df1.values
df2 = df2.values

In [0]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
X_train ,X_test, y_train,y_test  = train_test_split(df1,df2, test_size=0.2, random_state=200)
X_train, X_test = np.array(X_train, np.float32), np.array(X_test, np.float32)


In [0]:
cls = 2
size = 75*15
batchsize = 8
epochs =4000

In [0]:
# Standardize the dataset 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
#X_valid_scaled = scaler.transform(X_valid.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))



In [0]:
# =  X_train_scaled.reshape(X_train_scaled.shape[0],X_train_scaled.shape[1],1 )
#X_test_scaled =  X_test_scaled.reshape(X_test_scaled.shape[0],X_test_scaled.shape[1],1 )
#X_test_scaled.shape, X_train_scaled.shape
#X_test_scaled.shape[2], X_test_scaled.shape[1]

In [0]:
input_dim = X_train_scaled.shape[1]

feature = Input(shape = (input_dim, 1))
     
x = GaussianNoise(0.01)(feature)
x = Conv1D(filters= 128, kernel_size = 8, strides=4, padding='same', dilation_rate=1, 
            activation='relu', use_bias=True, kernel_initializer='glorot_uniform',
            bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, 
            activity_regularizer=None, kernel_constraint=None, bias_constraint=None,
            name = 'conv1D_1')(x)
x = BatchNormalization()(x)
#     x = PReLU()(x)
#x = MaxPooling1D(pool_size=2, strides=1, name = 'MP_1')(x)
#     x = Dropout(0.25)(x)
x = Flatten(name = 'flat_1')(x)
     
"""x_x = GaussianNoise(0.01)(feature)
x_x = Conv1D(filters= 64, kernel_size = 12, strides= 6, padding='same', dilation_rate=1, 
            activation='relu', use_bias=True, kernel_initializer='glorot_uniform',
            bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, 
            activity_regularizer=None, kernel_constraint=None, bias_constraint=None,
            name = 'conv1D_2')(x_x)
x_x = BatchNormalization()(x_x)
#     x_x = PReLU()(x_x)
#x_x = MaxPooling1D(pool_size=2, strides=2, name = 'MP_2')(x_x)
#     x_x = Dropout(0.25)(x_x)
x_x = Flatten()(x_x)"""
     
x_x_x = GaussianNoise(0.01)(feature)
x_x_x = Conv1D(filters= 32, kernel_size = 16, strides= 8, padding='same', dilation_rate=1, 
            activation='relu', use_bias=True, kernel_initializer='glorot_uniform',
            bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, 
            activity_regularizer=None, kernel_constraint=None, bias_constraint=None,
            name = 'conv1D_3')(x_x_x)
x_x_x = BatchNormalization()(x_x_x)
#     x_x_x = PReLU()(x_x_x)
#x_x_x = MaxPooling1D(pool_size=2, strides=2, name = 'MP_3')(x_x_x)
#     x_x_x = Dropout(0.25)(x_x_x)
x_x_x = Flatten()(x_x_x)
     
feature_f = GaussianNoise(0.01)(feature)
#     feature_f = MaxPooling1D(pool_size=4, strides=2, name = 'MP_4')(feature_f)
#     feature_f = Dropout(0.25)(feature_f)
feature_f = Flatten(name = 'flat_2')(feature_f)
     #
x = concatenate([x, x_x_x, feature_f])
     
x = Dense(256, activation = 'relu', name = 'dense_1')(x)
x = BatchNormalization()(x)
x = PReLU()(x)
     
x = Dense(128, activation = 'relu', name = 'dense_2')(x) #increase the dimension here for better speration in stage2 ?
x = BatchNormalization()(x)
x = PReLU()(x)
     
x = Dropout(0.5)(x)
pred = Dense(1, activation = 'sigmoid', name = 'dense_3')(x)
     
model = keras.Model(feature, pred)

     

checkpointer1 = ModelCheckpoint(filepath = 'model.weights.best.hdf5', monitor ='val_accuracy', verbose = 2, save_best_only= True)
checkpointer2 = EarlyStopping(monitor= 'val_accuracy', min_delta=0.0005, patience= 5, verbose = 2, mode = 'auto')

model.compile( optimizer= tf.keras.optimizers.Adam(lr =0.0001), loss= 'binary_crossentropy',  metrics=['accuracy'])
     
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)
history = model.fit(x=X_train_scaled, y=y_train,
                         batch_size = batchsize,
                         epochs = epochs, verbose = 0,
                         validation_split = 0.2,
#                         validation_data = (x_test_std, y_test),
                         callbacks=[checkpointer1,checkpointer2])
     

In [0]:
model.save_weights('model.weights.best.hdf5')